In [1]:
import json

import paho.mqtt.client as mqtt


def on_response_recieved(client, userdata, message):
    payload_str = message.payload.decode("utf-8")
    #print("received message: " , str(payload_str))
    payload = json.loads(payload_str)
    proxy._response = payload


class MQTTProxy():
    def __init__(self, broker):
        '''
        Initialize an MQTTProxy object.

        Parameters
        ----------
        broker : string
            Url for the MQTT broker.
        '''
        self._broker = broker
        self._client = mqtt.Client("OpenDrop Remote Proxy")
        self._client.connect(broker)
        
        self._client.loop_start()
        self._client.subscribe("opendrop/response")
        self._client.on_message=on_response_recieved
        self._response = {}
        
    def identify(self):
        # Return a string that uniquely identifies the OpenDrop.
        # The string is of the form "GaudiLabs,<model>,<serial number>,<software revision>" .
        self._client.publish("opendrop/command",
                             json.dumps({'command': 'identify'}))
        while not self._response:
            pass
        response = self._response
        self._response = {}
        return response

    def wait_for_response(self):
        # TODO: fix this hacky way to wait for a response
        while not self._response:
            pass
        response = self._response
        self._response = {}
        return response
    
    def set_voltage(self, voltage):
        self._client.publish("opendrop/command",
                             json.dumps({'command': 'set_voltage', 'args': [voltage]}))

    @property
    def voltage(self,):
        self._client.publish("opendrop/command",
                             json.dumps({'command': 'voltage', 'type': 'property'}))
        return self.wait_for_response()
    
    @property
    def state_of_channels(self,):
        self._client.publish("opendrop/command",
                             json.dumps({'command': 'state_of_channels', 'type': 'property'}))
        return self.wait_for_response()

In [2]:
# Connect to an mqtt broker
proxy = MQTTProxy("localhost")

In [3]:
proxy.identify()

'GaudiLabs,OpenDrop,#00,v3.2'

In [4]:
proxy.voltage

100.0

In [ ]:
proxy.state_of_channels